In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/NLP/dataset/text

/content/drive/MyDrive/NLP/dataset/text


In [5]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.0MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
     |████████████████████████████████| 901kB 48.1MB/s 


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertModel, BertConfig

import numpy as np
import pickle

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
with open('xtrain.pkl', 'br') as fr:
    xtrain = pickle.load(fr)

with open('ytrain.pkl', 'br') as fr:
    ytrain = pickle.load(fr)

In [9]:
bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese')

In [10]:
class DocCls(nn.Module):
    def __init__(self, bert):
        super(DocCls, self).__init__()
        self.bert = bert
        self.cls = nn.Linear(768, 9)
    def forward(self, x):
        bout = self.bert(x)
        bs = len(bout[0])
        h0 = [bout[0][i][0] for i in range(bs)]
        h0 = torch.stack(h0, dim=0)
        return self.cls(h0)

In [11]:
net = DocCls(bert).to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [12]:
net.train()
for ep in range(30):
    lossK = 0.0
    for i in range(len(xtrain)):
        x = torch.LongTensor(xtrain[i]).unsqueeze(0).to(device)
        y = torch.LongTensor([ytrain[i]]).to(device)
        out = net(x)
        loss = criterion(out,y)
        lossK += loss.item()
        if (i % 500 == 0):
            print(ep, i, lossK)
            lossK = 0.0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
outfile = 'doccls-' + str(ep) + '.model'
torch.save(net.state_dict(), outfile)

0 0 2.5443687438964844
0 500 687.9288250803947
1 0 1.0273144245147705
1 500 238.5222045588307
2 0 0.2862291634082794
2 500 112.48586988425814
3 0 0.11778568476438522
3 500 60.9156981658889
4 0 0.07241839915513992
4 500 26.768006582919043
5 0 0.02829180657863617
5 500 15.830688473477494
6 0 0.00950702466070652
6 500 6.785633721447084
7 0 0.01733485609292984
7 500 5.998843508277787
8 0 0.017177261412143707
8 500 4.657191421560128
9 0 0.011151960119605064
9 500 4.583084108147887
10 0 0.0027305721305310726
10 500 2.723591085843509
11 0 0.0027795510832220316
11 500 2.269234418083215
12 0 0.0022093667648732662
12 500 2.469739808628219
13 0 0.005376878194510937
13 500 1.7153163072362076
14 0 0.001823549740947783
14 500 1.5605244763428345
15 0 0.002137520583346486
15 500 1.3725689506609342
16 0 0.004478306509554386
16 500 1.4947719205301837
17 0 0.0013091336004436016
17 500 1.2596090924052987
18 0 0.00242380378767848
18 500 1.3163928622743697
19 0 0.0013743727467954159
19 500 1.105493660521460

#推論

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertModel, BertConfig

import numpy as np
import pickle
import sys

In [15]:
argvs = sys.argv
argc = len(argvs)

In [16]:
argvs

['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py',
 '-f',
 '/root/.local/share/jupyter/runtime/kernel-8fbe26b6-c7ba-4a66-82e7-e85051940551.json']

In [17]:
config = BertConfig.from_pretrained('cl-tohoku/bert-base-japanese')
bert = BertModel(config=config)

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
with open('xtest.pkl','br') as fr:
    xtest = pickle.load(fr)

with open('ytest.pkl','br') as fr:
    ytest = pickle.load(fr)

In [20]:
class DocCls(nn.Module):
    def __init__(self,bert):
        super(DocCls, self).__init__()
        self.bert = bert
        self.cls=nn.Linear(768,9)
    def forward(self,x):
        bout = self.bert(x)
        bs = len(bout[0])
        h0 = [ bout[0][i][0] for i in range(bs)]
        h0 = torch.stack(h0,dim=0)
        return self.cls(h0)

In [22]:
net = DocCls(bert).to(device)
net.load_state_dict(torch.load('/content/drive/MyDrive/NLP/dataset/text/doccls-29.model'))

<All keys matched successfully>

In [23]:
real_data_num, ok = 0, 0
net.eval()
with torch.no_grad():
    for i in range(len(xtest)):
        x = torch.LongTensor(xtest[i]).unsqueeze(0).to(device)
        ans = net(x)
        ans1 = torch.argmax(ans,dim=1).item()
        if (ans1 == ytest[i]):
            ok += 1
        real_data_num += 1
print(ok, real_data_num, ok/real_data_num)

661 738 0.8956639566395664
